# Deep Learning - Exercise 1
In this lecture, we'll explore essential TensorFlow 2 and Keras concepts through hands-on examples with the MNIST dataset - the "Hello World" of deep learning. We'll cover:

**Core Concepts**
- 🚀 Building and training a basic neural network for digit classification
- 📒 Understanding validation strategies for model evaluation
- 📊 Exploring model complexity and its impact on performance
- ✅ Designing optimal architectures using fully connected layers

![meme01](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_meme_01.jpg?raw=true "AI Meme")

[Open in Google colab](https://colab.research.google.com/github/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_01.ipynb)

[Download from Github](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/dl_01.ipynb)

##### Remember to set **GPU** runtime in Colab!

### Import of the TensorFlow
The main version of the TensorFlow (TF) is a in the Version package in the field VERSION
Since the TensformFlow 2.0 everything was encapsulaed under the KERAS api.

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import accuracy_score

tf.version.VERSION

# Import a dataset
* Datasets are stored in the keras.datasets submodule. Few testing datasets are stored here and installed together with the TF package

In [ ]:
# mnist is the basic dataset with handwritten digits
mnist = tf.keras.datasets.mnist

# data from any dataset are loaded using the load_Data function
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# the data are in the form of 28x28 pixes with values 0-255.
print('Train data shape: ', x_train.shape)
print('Test data shape:  ', x_test.shape)

* The dataset consists of 60,000 training images and 10,000 testing images. All of the are 28x28 pixels. 

## Let's take look on the data
* 💡 Look closely on the value scale - it is from 0 to 255 as usual in grayscale images.

## We can see that the numbers are already centered - you won't see this feature in the real world images often 🙂
* Some of the images are quite easy to classify but on the other hand there is a lot of noise in the data as well.

**Example Analysis**
- Some digits exhibit characteristics of multiple classes
- **Sample 1**: Clear, well-defined digit with distinct features
- **Sample 2**: Ambiguous sample showing characteristics of both "1" and "7"
    - Missing the horizontal stroke typical of "7"
    - Slight angle resembling a slanted "1"

In [ ]:
plt.figure()
plt.imshow(x_train[12], cmap='gray', vmin = 0, vmax = 255)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.figure()
plt.imshow(x_train[42], cmap='gray', vmin = 0, vmax = 255)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
y_train[42]

## Normalizing Pixel Values

- Convert pixel values from 0-255 to \(0,1\) by dividing each value by 255.
- **🔎 Why Normalize?**
  - Smaller input values stabilize gradient updates.
  - Consistent input scale improves the adjustment of network weights.

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

# Make better visualization of the data to better understand how complex they are
* You can see that "noisy" digit and different handwriting style is really no exception

In [ ]:
class_names = [str(x) for x in range(10)]

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

## 🌳 Before we dive in the deep learning territory, let's try to create some baseline model using a machine learning so we can compare the approaches
* 🔎 Are ML models capable of processing image data?
    * How to deal with a matrix input?

In [ ]:
alg = DecisionTreeClassifier()
alg.fit(x_train.reshape(-1, 28*28), y_train)

In [ ]:
y_pred = alg.predict(x_test.reshape(-1, 28*28))

In [ ]:
y_pred

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_pred)

# We can see that we were able to create a classifier very easily.
* 💡 However it is no secret, that DL models accuracy on MNIST can be >= 98% even with simple architecture
    * Given the fact, our result is not very impresive 🙂

# Basic model - ANN with very simple hierarchy
   * Model is created using layers, many layers exists in the [layer submodule](https://www.tensorflow.org/api_docs/python/tf/keras/layers)
   * Each layer uses a activation functions collected in the [module nn](https://www.tensorflow.org/api_docs/python/tf/nn)
 
### 💡 There are 2 ways of using the Keras API - **Sequential** and **Functional**
   * We will start with the sequential one

* 🔎 Why do we use activation functions?
* 🔎 How is ANN different from lin. regression?

# 🚀 Let's design our first ANN
   * Can you draw such network on a table?

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),# Flatten module flatten the multidimension input into single vector 28x28 = 784 float numbers
    keras.layers.Dense(32, activation=tf.nn.relu), # standard dense-fully connected layer with the rectified lineaar function as an activation
    keras.layers.Dense(10, activation=tf.nn.softmax), # another fully-connected layer with softmax activation function
])

model.summary() # prints the summary of the model

### 🔎 What is the meaning of the *Total params* number?

# 📒 Model Compilation

Each model must be compiled to fit the data and predict labels.

## ⚙️ Optimizers

- A variety of optimizers are available, many based on gradient descent, with gradient-free options (e.g., [Nevergrad by META](https://github.com/facebookresearch/nevergrad)).

### 🔑 Selected Optimizers

- **📉 Gradient Descent**  
  Uses the full dataset; not ideal for large datasets.
  
- **🎲 Stochastic Gradient Descent (SGD)**  
  Estimates the gradient using a subset of data.
  
- **📊 RMSProp**  
  Adapts the learning rate using the running average of recent gradients.
  
- **⚡ ADAM**  
  Combines gradient average and second moment to adapt the learning rate.

## 🧮 Loss Functions

💡 *Training aims to optimize weights by minimizing a loss function. The choice depends on the task and network architecture.*

- **📐 Mean Squared Error (MSE)** – For regression tasks (logarithmic variant available).
- **📏 Mean Absolute Error (MAE)** – Uses absolute differences instead of squared values.
- **🔢 Binary Cross-Entropy** – For binary tasks; requires a sigmoid activation function.
- **🌈 Categorical Cross-Entropy** – For multi-class tasks; requires softmax and one-hot encoded labels.
- **🌟 Sparse Categorical Cross-Entropy** – Similar to categorical loss but works with integer labels.

## 📊 Metrics

💡 *Metrics evaluate model performance. Choose metrics based on the task and label distribution.*

### 📈 Regression Metrics
- Mean Squared Error (MSE)
- Mean Absolute Error (MAE)
- Mean Absolute Percentage Error (MAPE)

### 🧩 Classification Metrics
- Binary Accuracy
- Categorical Accuracy
- Sparse Categorical Accuracy
- Top-k Categorical Accuracy
- Sparse Top-k Categorical Accuracy

⚡ *Remember*: Accuracy may not always be the best metric, especially for imbalanced datasets. Consider alternatives like F1-score or recall in such cases.


# 🚀 Model Compilation

- **Accuracy Metric:** We evaluate our model using the accuracy metric.
- **Loss Function:** We use `SparseCategoricalCrossentropy` because our labels are provided as integers.

## Dense Layer Output Options

- **Probabilities:**  
  The output is passed through a softmax function, producing a normalized probability distribution over the classes (sums to 1).
  
- **Logits:**  
  The layer produces raw activations, which are not normalized.

## Configuring `from_logits`

Your loss function needs to know what kind of output to expect:
- 💡 If the output layer uses a **softmax** activation, set `from_logits` to `False`.
- 💡 If the output layer does **not** use softmax, set `from_logits` to `True`.

In [ ]:
model.compile(optimizer='adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy'],
    )

## 📊 Model visualization
* The model may be printed into image like the following image of our model:

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

# 🚀 Now we can fit the model to the input data
* The `fit()` method fit the model to the data, the parameters are *data* and *labels* from the train set and number of *epoch* to be trained.
* The `validation_split` parameter is also very common. What does the parameter do?
    * What is the validation set? How is it different from the test set?

## FAQ section 🙂
* How does the ANN training procedure look like?
* What is the *epoch*?
* What is the *batch*?
* Why do we use batches? 
    * Is possible to have batch of size `len(data)` or of size `1`? 
    * What are caveates of these cases?

# Define the callbacks
* 💡  Always use `ModelCheckpoint` callback so you overcome the possible overfitting in the last few epochs!

## The `.fit()` API is pretty powerful
* It is common to use some sort of a callback, we will use `ModelCheckpoint` callback which saves the best weights configuration obtained during training so the overfitting at the final phase of training will be suppressed

* **💡 The best weights are determined using the validation loss value**

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

# Fit the model

In [ ]:
history = model.fit(x_train, y_train, validation_split=0.2, epochs=3, callbacks=[model_checkpoint_callback], batch_size=128)

# 📈 Plot Analysis: Accuracy and Loss
The plot displays both training and validation trends, with accuracy increasing and loss decreasing over epochs.

- **Lower Loss & Higher Accuracy:**  
  It's normal for the loss to decrease and accuracy to increase during training, which indicates that the model is learning. Occasional fluctuations—where the loss may spike briefly—are also common due to the stochastic nature of optimization.

- **Loss Function Behavior:**  
  The loss can sometimes increase in short intervals because of noise in the training process, but overall a downward trend is desired.

- **Overfitting Indicator:**  
  If the training loss continues to decrease while the validation loss starts increasing, this situation is known as **overfitting**. Overfitting occurs when the model learns the training data too well and fails to generalize to unseen data.

In [ ]:
plt.figure()
for key in history.history.keys():
    plt.plot(history.epoch, history.history[key], label=key)
plt.legend()
plt.tight_layout()

# 📌 Beware that ANNs are prone to overfitting!

### How does overfit look like?
* Y axis = Accuracy
* X axis = Epoch

![overfit](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/overfit_acc.png?raw=true "Overfit")

# 🔎 How can we solve the overfitting issue?

## 1 - Use simpler model
* The model often overfit because there is a lot of parameters for the amount of input data

## 2 - Use Dropout layer
* Node sampling instead of edge sampling 
* If a node is dropped, then all incoming and outgoing connections from that node need to be dropped as well
* We sample sub-networks from the original one - basically ensemble of networks
* There won't be some "alpha" node in the individual layer with huge weight coefficient
    * Responsibility for prediction will be shared among multiple nodes

![dropout](https://github.com/rasvob/2020-21-ARD/raw/master/images/dropout.jpeg "Dropout")

## 💡 Keras API
- `keras.layers.Dropout(0.5)`
- Take a look [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)

## Beware that the best weights needs to be loaded after the training is finished!
* **💡 Otherwise you use the weights from the last epoch!**

In [ ]:
model.load_weights("best.weights.h5")

## Now we can use *evaluate()* function for obtaining the accuracy using the test set

![meme02](https://github.com/rasvob/VSB-FEI-Deep-Learning-Exercises/blob/main/images/dl_meme_02.jpg?raw=true "AI Meme 02")

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy: ', test_acc)

# Can we obtain the labels and use it for our own evaluation without Keras? Sure!

## We will obtain the raw softmax outputs
* 🔎 What is the range of the vector values?
* 🔎 Do they sum-up to some number?

In [ ]:
y_pred_proba = model.predict(x_test)

In [ ]:
y_pred_proba[0]

In [ ]:
np.sum(y_pred_proba[0])

## Now we can extract the index of the highest probability and get the labels

In [ ]:
y_pred = np.argmax(y_pred_proba, axis=1)

In [ ]:
y_pred

# Is the model any better than the ML baseline?

In [ ]:
accuracy_score(y_true=y_test, y_pred=y_pred)

# We can now re-build the same network using a Functional API
* You can use any API that you like however I recommend the **Functional** one as it is more versatile in advanced use-cases, e.g. building ANN architecture with skip-connect synaptic links

In [ ]:
ann_input = keras.layers.Input(shape=(28, 28))
flatten = keras.layers.Flatten()(ann_input)
hidden = keras.layers.Dense(32, activation='relu')(flatten)
ann_output = keras.layers.Dense(10, activation='softmax')(hidden)

model = keras.Model(ann_input, ann_output)

model.compile(optimizer='adam',
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy'],
    )

model.summary()

## ⚡ The rest of the training process is the same

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best.weights.h5',
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)

history = model.fit(x_train, y_train, validation_split=0.2, epochs=3, callbacks=[model_checkpoint_callback], batch_size=128)
model.load_weights("best.weights.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy: ', test_acc)

# Inference and Deployment

- **Save the Model:**  
  📌 Once training is complete, save your model to a file. This avoids retraining every time you need to make predictions.

- **Load the Model for Inference:**  
  📂 In your web app, load the saved model and use it to process incoming data and generate predictions.

## Save the model
* We are using the new `.keras` format

In [ ]:
model.save('mnist_model.keras')

## Load the model back to memory and test it

In [ ]:
loaded_model = keras.models.load_model('mnist_model.keras')
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy: ', test_acc)

# ✅ Task (2p)
1) Experiment with batch_sizes, use these two settings and compare the results (📌 Compare training time and accuracy) - **(0.5p)**

    1) Use batch_size=1

    2) Use batch_size=1024
    
2) Add `Dropout` layer just before the `SoftMax` output layer in you architecture use these two settings and compare the results (📌 Take a look at the train and val accuracy) - **(0.5p)**

    1) Dropout(0.2)
    
    2) Dropout(0.95)
    
3) Define your own architecture using either one of the showed API and try to beat the basic model - **(1p)**